In [2]:
!pip3 install shapely geopandas geo



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# get cordinates - https://www.latlong.net/|
db_ip="localhost"
db_user="root"
db_pass="new_password"
db_name="p3wifi"

### Target: get Country Wifi map for specific country from public sources

#### Database connection

In [4]:
import mysql.connector

# Step 1: Retrieve data from MySQL database 
conn = mysql.connector.connect(
    host=f'{db_ip}',
    user=f'{db_user}',
    password=f'{db_pass}',
    database=f'{db_name}',
    charset='utf8',
    collation='utf8mb4_general_ci'  # Set a compatible collation
)
if conn.is_connected():
    print("Connected to MySQL database")

# [longitude, latite]
CZECH = {"latitude":[48.474378,51.045193], "longitude":[12.068481,18.860779]}

Connected to MySQL database


#### SQL database -> CSV

In [5]:
# data saved to rectangle aproimation to ../inputs/CZEC_RECT.csv with
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(
    f'mysql+mysqlconnector://{db_user}:{db_pass}@{db_ip}/{db_name}',
    connect_args={'charset': 'utf8', 'collation': 'utf8mb4_general_ci'}
)

with open('./get_3wifi_data.sql', 'r') as file:
    sql_script = file.read()
df = pd.read_sql_query(sql_script, engine)

df.to_csv('../../data/clean/CZECH2_RECT.csv', index=False)

In [13]:
import pandas as pd
from shapely.geometry import Point, Polygon


import geopandas as gpd
from shapely.geometry import Point

# Read the GeoJSON file
gdf = gpd.read_file('../../data/raw/geoBoundaries-CZE-ADM0.geojson')

# Extract the polygon
polygon = gdf.geometry.iloc[0]

# Read the CSV data
csv_data = pd.read_csv('../../data/clean/CZECH2_RECT.csv')
def in_polygon(lon, lat):
    point = Point(lon, lat)
    return polygon.contains(point)

#Get Wifi passlist, not unique, not sorted
with open('../../data/clean/Wifi_passlist.txt', 'w') as outfile:
    for WifiKey, lon, lat in zip(csv_data['WifiKey'], csv_data['longitude'], csv_data['latitude']):
        if WifiKey not in {"<empty>", "nan","<not accessible>"} and in_polygon(lon, lat):
            print(WifiKey, file=outfile)